In [7]:
import CrawlingSearchOECD
import selenium
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy
from CrawlingSearchOECD import webdriver
from CrawlingSearchOECD import By
from CrawlingSearchOECD import chrome_options
from CrawlingSearchOECD import chrome_service
import proxy_randomizer
from proxy_randomizer import getProxList
import random
from bs4 import BeautifulSoup

In [8]:
def get_dataframe_from_webPage(url,sleepTime):
    Main_page=webdriver.Chrome(options=chrome_options,service=chrome_service)
    Main_page.get(url)
    try:
        element_present = EC.presence_of_element_located((By.ID, '_ctl0_ContentPlaceHolder1__ctl4_contentcenter'))
        WebDriverWait(Main_page, sleepTime).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        print("Page loaded")



    TableName,df=fetch_dataframe(Main_page)
    Main_page.close()

    return TableName,df

In [9]:
def fetch_dataframe(Main_page):
    Table_title=Main_page.find_element(By.CLASS_NAME,'dtitle')
    if ("indicators" in Table_title.text):
        TableName=Main_page.find_element(By.XPATH,'//*[@id="PDim_INDICATOR"]/option[1]').text
    else:
        try:
            TableName=Main_page.find_element(By.XPATH,'//*[@id="PDim_VARIABLE"]/option[1]').text
        except NoSuchElementException:
            TableName=Main_page.find_element(By.XPATH,'//*[@id="QueryNameText"]').text


    DataTable=Main_page.find_element(By.XPATH,'//*[@id="tabletofreeze"]/table/thead')
    tr_arr=DataTable.get_property('children')
    TableHead=tr_arr[-4]
    colParamList=list(['Country'])
    dictionary={'Country':[]}
    ColParamRange=TableHead.find_elements(By.CLASS_NAME,'HDim')

    for item in ColParamRange:
        colParamList.append(item.text)
        colName=item.text
        dictionary.update({colName:[]})

    TableBody=Main_page.find_element(By.XPATH,'//*[@id="tabletofreeze"]/table/tbody')
    TableRows=TableBody.get_property('children')
    
    i=1
    for row in TableRows:
        if(i % 2 !=0):
            t=(row.find_element(By.CLASS_NAME,'RowDimLabel').text)
            dictionary[colParamList[0]].append(t)
            i+=1
        elif(i % 2 ==0):
            t=(row.find_element(By.CLASS_NAME,'RowDimLabel2').text)
            dictionary[colParamList[0]].append(t)
            i+=1

    i=1
    for row in TableRows:
        if (i % 2!=0):
            cells=row.find_elements(By.CLASS_NAME,'Data')
            i+=1
        elif(i % 2 ==0):
            cells=row.find_elements(By.CLASS_NAME,'Data2')
            i+=1
        j=1
        for cell in cells:
            dictionary[colParamList[j]].append(cell.get_property('textContent'))
            j+=1

    df=pd.DataFrame(dictionary)

    return TableName,df

In [10]:
def get_additional_Table_url_from_WebPage(url,diffrentDataset=None,sleepTime="10",scroll="0"):
    Main_page=webdriver.Chrome(options=chrome_options,service=chrome_service)
    Main_page.get(url)
    time.sleep(sleepTime)
    Table_title=Main_page.find_element(By.CLASS_NAME,'dtitle')
    if ("indicators" in Table_title.text):
        TableName=Main_page.find_element(By.XPATH,'//*[@id="PDim_INDICATOR"]/option[1]').text
    else:
        TableName=Main_page.find_element(By.XPATH,'//*[@id="PDim_VARIABLE"]/option[1]').text
    ContentTable=Main_page.find_element(By.XPATH,'//*[@id="BrowseThemesContent"]')
    if (diffrentDataset):
        action = ActionChains(Main_page)
        Table_to_close=ContentTable.find_element(By.CLASS_NAME,'opened')
        action.move_to_element_with_offset(Table_to_close,5,0)
        action.click()
        action.perform()
        Table_arr=ContentTable.find_elements(By.TAG_NAME,'li')
        for table in Table_arr:
            if (table.text==diffrentDataset):
                table.click()
                break
    OpenedTable=ContentTable.find_element(By.CLASS_NAME,'opened')
    options=OpenedTable.find_elements(By.CLASS_NAME,'q')
    addTableNames=list()
    addDataFrames=list()
    for option in options:
        if (option.text!=TableName):
            option.click()
            time.sleep(sleepTime)
            Main_page.execute_script("var q=document.documentElement.scrollTop="+scroll) 
            Tname,df=fetch_dataframe(Main_page)
            addTableNames.append(Tname)
            addDataFrames.append(df)
                           
    Main_page.close()
    return  addTableNames,addDataFrames

In [11]:
chrome_options=Options()
chrome_options.add_argument("--incognito")

In [12]:
sleepTime=10
main_url=CrawlingSearchOECD.GetDatasetUrl("Road accidents")
roadAccidents_T_Name,roadAccidentsDf=get_dataframe_from_webPage(main_url,sleepTime=sleepTime)
roadAccidentsDf.to_csv('./OECD_Data/'+roadAccidents_T_Name+'.csv')
addTableNames,addDataFrames=get_additional_Table_url_from_WebPage(main_url,None,sleepTime)
if (addTableNames):
    i=0
    for table in addDataFrames: 
       table.to_csv('./OECD_Data/'+addTableNames[i]+'.csv')
       i+=1

The latest publicated year is:2021 [2]
The title of the reaserch is:
Road accidents (Edition 2020) Dataset Archive English
09 Mar 2021 International Transport Forum in ITF Transport Statistics
The Phrase is found
Heare are another options of Datasets:
                                            Datasets  \
0                                    Goods transport   
1                                Passenger transport   
2                                     Road accidents   
3                    Short-term transport indicators   
4  Transport infrastructure investment and mainte...   
5                   Transport performance indicators   
6                     Road haulage charges and taxes   
7                    Casualties by age and road user   
8                             Urban access framework   
9                        Annual transport statistics   

                                                 URL  
0  https://www.oecd-ilibrary.org/deliverdotstat?i...  
1  https://www.oecd-i

In [13]:
sleepTime=10
url=CrawlingSearchOECD.GetDatasetUrl("Transport infrastructure investment and maintenance")
Investment_T_Name,InvestmentDf=get_dataframe_from_webPage(url,sleepTime)
InvestmentDf.to_csv('./OECD_Data/'+Investment_T_Name+'.csv')
addTableNames,addDataFrames=get_additional_Table_url_from_WebPage(url,None,sleepTime)
if (addTableNames):
    i=0
    for table in addDataFrames:
       table.to_csv('./OECD_Data/'+addTableNames[i]+'.csv')
       i+=1

The latest publicated year is:2019 [2]
The title of the reaserch is:
Transport infrastructure investment and maintenance (Edition 2019) Dataset Archive English
26 Nov 2019 International Transport Forum in ITF Transport Statistics
The Phrase is found
Heare are another options of Datasets:
                                            Datasets  \
0                                    Goods transport   
1                                Passenger transport   
2                                     Road accidents   
3                    Short-term transport indicators   
4  Transport infrastructure investment and mainte...   
5                   Transport performance indicators   
6                     Road haulage charges and taxes   
7                    Casualties by age and road user   
8                             Urban access framework   
9                        Annual transport statistics   

                                                 URL  
0  https://www.oecd-ilibrary.org/delive

In [14]:
sleepTime=10
url=CrawlingSearchOECD.GetDatasetUrl("Passenger transport")
Investment_T_Name,InvestmentDf=get_dataframe_from_webPage(url,sleepTime)
InvestmentDf.to_csv('./OECD_Data/'+Investment_T_Name+'.csv')
addTableNames,addDataFrames=get_additional_Table_url_from_WebPage(url,None,sleepTime)
if (addTableNames):
    i=0
    for table in addDataFrames:
       table.to_csv('./OECD_Data/'+addTableNames[i]+'.csv')
       i+=1

The latest publicated year is:2021 [36]
The title of the reaserch is:
Passenger transport (Edition 2020) Dataset Archive English
09 Mar 2021 International Transport Forum in ITF Transport Statistics
The Phrase is found
Heare are another options of Datasets:
                                            Datasets  \
0                                    Goods transport   
1                                Passenger transport   
2                                     Road accidents   
3                    Short-term transport indicators   
4  Transport infrastructure investment and mainte...   
5                   Transport performance indicators   
6                     Road haulage charges and taxes   
7                    Casualties by age and road user   
8                             Urban access framework   
9                        Annual transport statistics   

                                                 URL  
0  https://www.oecd-ilibrary.org/deliverdotstat?i...  
1  https://www.

In [15]:
sleepTime=10
url=CrawlingSearchOECD.GetDatasetUrl("Annual transport statistics")
Investment_T_Name,InvestmentDf=get_dataframe_from_webPage(url,sleepTime)
InvestmentDf.to_csv('./OECD_Data/'+Investment_T_Name+'.csv')
addTableNames,addDataFrames=get_additional_Table_url_from_WebPage(url,None,sleepTime,"400")
if (addTableNames):
    i=0
    for table in addDataFrames:
       table.to_csv('./OECD_Data/'+addTableNames[i]+'.csv')
       i+=1

The latest publicated year is:2021 [1]
The title of the reaserch is:
Annual transport statistics Dataset English
21 Jul 2021 OECD in ITF Transport Statistics
The Phrase is found
Heare are another options of Datasets:
                                            Datasets  \
0                                    Goods transport   
1                                Passenger transport   
2                                     Road accidents   
3                    Short-term transport indicators   
4  Transport infrastructure investment and mainte...   
5                   Transport performance indicators   
6                     Road haulage charges and taxes   
7                    Casualties by age and road user   
8                             Urban access framework   
9                        Annual transport statistics   

                                                 URL  
0  https://www.oecd-ilibrary.org/deliverdotstat?i...  
1  https://www.oecd-ilibrary.org/deliverdotstat?i...  
2

In [16]:
sleepTime=10
url=CrawlingSearchOECD.GetDatasetUrl("Transport performance indicators")
Investment_T_Name,InvestmentDf=get_dataframe_from_webPage(url,sleepTime)
InvestmentDf.to_csv('./OECD_Data/'+Investment_T_Name+'.csv')
addTableNames,addDataFrames=get_additional_Table_url_from_WebPage(url,None,sleepTime)
if (addTableNames):
    i=0
    for table in addDataFrames:
       table.to_csv('./OECD_Data/'+addTableNames[i]+'.csv')
       i+=1

The latest publicated year is:2019 [2]
The title of the reaserch is:
Transport performance indicators (Edition 2019) Dataset Archive English
26 Nov 2019 International Transport Forum in ITF Transport Statistics
The Phrase is found
Heare are another options of Datasets:
                                            Datasets  \
0                                    Goods transport   
1                                Passenger transport   
2                                     Road accidents   
3                    Short-term transport indicators   
4  Transport infrastructure investment and mainte...   
5                   Transport performance indicators   
6                     Road haulage charges and taxes   
7                    Casualties by age and road user   
8                             Urban access framework   
9                        Annual transport statistics   

                                                 URL  
0  https://www.oecd-ilibrary.org/deliverdotstat?i...  
1  

In [17]:
sleepTime=15
url=CrawlingSearchOECD.GetDatasetUrl("Road accidents")
addTableNames,addDataFrames=get_additional_Table_url_from_WebPage(url,"Casualties by age and road user",sleepTime,"400")
if (addTableNames):
    i=0
    for table in addDataFrames:
       table.to_csv('./OECD_Data/'+addTableNames[i]+'.csv')
       i+=1

The latest publicated year is:2021 [2]
The title of the reaserch is:
Road accidents (Edition 2020) Dataset Archive English
09 Mar 2021 International Transport Forum in ITF Transport Statistics
The Phrase is found
Heare are another options of Datasets:
                                            Datasets  \
0                                    Goods transport   
1                                Passenger transport   
2                                     Road accidents   
3                    Short-term transport indicators   
4  Transport infrastructure investment and mainte...   
5                   Transport performance indicators   
6                     Road haulage charges and taxes   
7                    Casualties by age and road user   
8                             Urban access framework   
9                        Annual transport statistics   

                                                 URL  
0  https://www.oecd-ilibrary.org/deliverdotstat?i...  
1  https://www.oecd-i